In [8]:
import sys
sys.path.append(sys.path[0][:sys.path[0].rfind('/')])
from StabilizerCircuits import *

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
from random import random
import copy
from tqdm import tqdm
from multiprocessing import Pool

ModuleNotFoundError: No module named 'StabilizerCircuits'

# Error params

In [36]:
T1 = 30 # Qubit relaxation time
Tf = 60 # Qubit dephasing time
Tg1Q = 0.02 # Single-qubit gate time
Tg2Q = 0.04 # Two-qubit gate time
tm = 0.3 # Measurement time
td = 0.3 # Depletion time


p_axis = 10**(-4) # In-axis rotation error
p_plane = 5 * 10**(-4) # In-plane rotation error 

dphi = 0.01

eta_origin = np.array([[0.9985, 0, 0.0015, 0],
                       [0.005, 0.0015, 0.0149, 0.9788]])

eta = np.zeros((2, 3))
for i in range(0, 3):
    eta[:, i] = np.sum(eta_origin[:, :i+1], axis = 1)
eta


#Hadamard_noise:
ph = (1-np.exp(dphi**2/2))/(2-np.sqrt(2))
# ph = 0

# ZZ_noise
pZZ = (1-np.exp(dphi**2/2))/2

#measure
p0 = eta_origin[0, 2] + eta_origin[0, 3]
p1 = eta_origin[1, 0] + eta_origin[1, 1]

In [37]:
# Tg1Q = 0.0 # Single-qubit gate time
# Tg2Q = 0.0 # Two-qubit gate time
# tm = 0 # Measurement time
# td = 0 # Depletion time

# p_axis = 0
# p_plane = 0

# #Hadamard_noise:
# ph = 0

# # ZZ_noise
# pZZ = 0

# #measure
# p0 = 0
# p1 = 0

# QEC cycle

In [38]:
def cycle(table, n, basis):
    # basis = '0', '1, '+', '-', 'i+', 'i-'
    if (n - 10)%8 != 0:
        print("Wrong n!!!")
    m_data = np.zeros((n//2), dtype=np.int8)
    m_ancilla = np.zeros((n//2), dtype=np.int8)
    
    even = np.arange(n//2)*2 # ancilla qubits
    odd = np.arange(n//2)*2 + 1 # data qubits
    
    # correction cycle
    for i in even:
        table = amplitude_damping(table, i, Tg1Q/2, T1)
        table = depolarize(table, i, p_plane, p_axis, p_plane)
        table = Hadamard_noise(table, i, ph)
    for i in even:
        table = amplitude_damping(table, i, Tg1Q/2 + Tg2Q/2, T1)
        table = amplitude_damping(table, i+1, Tg1Q + Tg2Q/2, T1)
        table = iSWAP(table, i, i+1)
        table = ZZ_noise(table, i, i+1, pZZ)
    for i in even:
        table = amplitude_damping(table, i, Tg1Q/2 + Tg2Q/2, T1)
        table = Phase(table, i)
        table = depolarize(table, i, p_plane, p_axis, p_plane)
        table = Hadamard_noise(table, i, ph)
    for i in odd:
        table = amplitude_damping(table, i, Tg1Q + Tg2Q, T1)
        table = amplitude_damping(table, i+1, Tg1Q/2 + Tg2Q/2, T1)
        table = iSWAP(table, i, (i+1)%n)
        table = ZZ_noise(table, i, (i+1)%n, pZZ)
    for i in even:
        table = amplitude_damping(table, i, Tg2Q, T1)
        table = amplitude_damping(table, i+1, Tg2Q, T1)
        table = iSWAP(table, i, i+1)
        table = ZZ_noise(table, i, i+1, pZZ)
    for i in even:
        table = amplitude_damping(table, i, Tg1Q/2 + Tg2Q/2, T1)
        table = Z_gate(table, i)
        table = depolarize(table, i, p_plane, p_axis, p_plane)
        table = Hadamard_noise(table, i, ph)
    for i in odd:
        table = amplitude_damping(table, i, Tg1Q + Tg2Q, T1)
        table = amplitude_damping(table, i+1, Tg1Q/2 + Tg2Q/2, T1)
        table = iSWAP(table, i, (i+1)%n)
        table = ZZ_noise(table, i, (i+1)%n, pZZ)
    for i in even:
        table = amplitude_damping(table, i, Tg2Q/2 + Tg1Q/2, T1)
        table = amplitude_damping(table, i+1, Tg2Q/2, T1)
        table = depolarize(table, i, p_plane, p_axis, p_plane)
        table = Hadamard_noise(table, i, ph)
        table = Phase(table, i+1)
        table = amplitude_damping(table, i, Tg1Q/2, T1)
        table = amplitude_damping(table, i+1, Tg1Q, T1)
        
        
        
    # ancilla measurement
    for i in even:
        table, m_ancilla[i//2] = measure(table, i)
        if m_ancilla[i//2] == -1:
            table = X_gate(table, i)
    
    # data measurement
    table_tmp = copy.copy(table)
    for i in odd:
        if basis == '+' or basis == '-':
            table_tmp = Ry_05_gate(table_tmp, i)
        elif basis == 'i+' or basis == 'i-':
            table_tmp = Rx05_gate(table_tmp, i)
        table_tmp, m_data[(i-1)//2] = measure(table_tmp, i)
    
    for i in range(n):
        table = amplitude_damping(table, i, tm+td, T1)
        
    return table, m_data.astype(np.bool), m_ancilla.astype(np.bool)

# MultiProcessing

In [76]:
N_process = 8
N = 10
fout = '_stab_instate'
N_run = 1000
N_cycle = 20
M_datanomeas = np.zeros((N_run, N_cycle, N//2), dtype = np.bool)
M_ancillanomeas = np.zeros((N_run, N_cycle, N//2), dtype = np.bool)
basis = '0'
table = init_state0(N)
odd = np.arange(N//2)*2 + 1
Meas = []


if basis == '1':
    for i in odd:
        table = X_gate(table, i)
elif basis == '+':
    for i in odd:
        table = Ry05_gate(table, i)
elif basis == '-':
    for i in odd:
        table = Ry_05_gate(table, i)
elif basis == 'i+':
    for i in odd:
        table = Rx_05_gate(table, i)
elif basis == 'i-':
    for i in odd:
        table = Rx05_gate(table, i)
        


def full_Cycle(proc_num, table=table, N=N, N_cycle=N_cycle):
    M_datanomeas = np.zeros((N_cycle, N//2), dtype = np.bool)
    M_ancillanomeas = np.zeros((N_cycle, N//2), dtype = np.bool)       
    for n_cycle in range(0, N_cycle):
        table, M_datanomeas[n_cycle], M_ancillanomeas[n_cycle] = cycle(table, N, basis)
    return [M_datanomeas, M_ancillanomeas]


with Pool(N_process) as p:
    Meas = list(tqdm(p.imap(full_Cycle, range(N_run)), total=N_run))

M_datanomeas = np.asarray(Meas, dtype=np.bool)[:,0]
M_ancillanomeas = np.asarray(Meas, dtype=np.bool)[:,1]


M_data = add_noise_to_measurements(M_datanomeas, p0, p1)
M_ancilla = add_noise_to_measurements(M_ancillanomeas, p0, p1)


for i in tqdm(range(N_run)):
    for j in range(N_cycle):
        M_ancillanomeas[i, j] = np.roll(M_ancillanomeas[i, j], 2*j)
        M_datanomeas[i, j] = np.roll(M_datanomeas[i, j], -2*j)
        M_ancilla[i, j] = np.roll(M_ancilla[i, j], 2*j)
        M_data[i, j] = np.roll(M_data[i, j], -2*j)

        
np.savez_compressed('N' + str(N) + fout + basis + '_' + str(N_run), M_data = M_data, M_ancilla = M_ancilla,\
                           M_datanomeas = M_datanomeas, M_ancillanomeas = M_ancillanomeas)

100%|██████████| 1000/1000 [00:01<00:00, 788.02it/s]


# Run

In [77]:
N = 10
fout = '_stab_instate'
N_run = 10
N_cycle = 20
M_datanomeas = np.zeros((N_run, N_cycle, N//2), dtype = np.bool)
M_ancillanomeas = np.zeros((N_run, N_cycle, N//2), dtype = np.bool)
basis = '0'
table = init_state0(N)
odd = np.arange(N//2)*2 + 1
Meas = []


if basis == '1':
    for i in odd:
        table = X_gate(table, i)
elif basis == '+':
    for i in odd:
        table = Ry05_gate(table, i)
elif basis == '-':
    for i in odd:
        table = Ry_05_gate(table, i)
elif basis == 'i+':
    for i in odd:
        table = Rx_05_gate(table, i)
elif basis == 'i-':
    for i in odd:
        table = Rx05_gate(table, i)
        


def full_Cycle(table=table, N=N, N_cycle=N_cycle):
    M_datanomeas = np.zeros((N_cycle, N//2), dtype = np.bool)
    M_ancillanomeas = np.zeros((N_cycle, N//2), dtype = np.bool)       
    for n_cycle in range(0, N_cycle):
        table, M_datanomeas[n_cycle], M_ancillanomeas[n_cycle] = cycle(table, N, basis)
    return [M_datanomeas, M_ancillanomeas]

for n_run in tqdm(range(N_run)):
    Meas.append(full_Cycle())

M_datanomeas = np.asarray(Meas, dtype=np.bool)[:,0]
M_ancillanomeas = np.asarray(Meas, dtype=np.bool)[:,1]


M_data = add_noise_to_measurements(M_datanomeas, p0, p1)
M_ancilla = add_noise_to_measurements(M_ancillanomeas, p0, p1)


for i in tqdm(range(N_run)):
    for j in range(N_cycle):
        M_ancillanomeas[i, j] = np.roll(M_ancillanomeas[i, j], 2*j)
        M_datanomeas[i, j] = np.roll(M_datanomeas[i, j], -2*j)
        M_ancilla[i, j] = np.roll(M_ancilla[i, j], 2*j)
        M_data[i, j] = np.roll(M_data[i, j], -2*j)

        
np.savez_compressed('N' + str(N) + fout + basis + '_' + str(N_run), M_data = M_data, M_ancilla = M_ancilla,\
                           M_datanomeas = M_datanomeas, M_ancillanomeas = M_ancillanomeas)

100%|██████████| 10/10 [00:00<00:00, 710.18it/s]
